In [1]:
import pandas as pd
data = pd.read_csv('titanic/train.csv')
print('Column names:\n {}\n\n'.format(data.columns))
print('Indexes:\n{}\n\n'.format(data.index))
print('Shape(rows,columns):\n{}\n\n'.format(data.shape))
print('Dimensions:\n{}\n\n'.format(data.ndim))
print('First rows:')
print(data.head())
print('Datatypes:{}'.format(data.dtypes))


OSError: File b'titanic/train.csv' does not exist

You can do basic manipulation of rows using +-/ and so on:

In [8]:
age_in_months = data['Age'] * 12
print(age_in_months.head())

0    264.0
1    456.0
2    312.0
3    420.0
4    420.0
Name: Age, dtype: float64


In [9]:
# We can remove the rows that are null:
cabin_is_null = data['Cabin'].isnull()
cabin_notnull = data['Cabin'][cabin_is_null == False]
print(cabin_notnull.head())

1      C85
3     C123
6      E46
10      G6
11    C103
Name: Cabin, dtype: object


In [11]:
# You could also remove this for all the data
data_not_null = data[cabin_is_null == False]
print(data_not_null.head())

# And then reset the index
data_not_null.reset_index(inplace=True)
print(data_not_null.head())

    PassengerId  Survived  Pclass  \
1             2         1       1   
3             4         1       1   
6             7         0       1   
10           11         1       3   
11           12         1       1   

                                                 Name     Sex   Age  SibSp  \
1   Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
3        Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
6                             McCarthy, Mr. Timothy J    male  54.0      0   
10                    Sandstrom, Miss. Marguerite Rut  female   4.0      1   
11                           Bonnell, Miss. Elizabeth  female  58.0      0   

    Parch    Ticket     Fare Cabin Embarked  
1       0  PC 17599  71.2833   C85        C  
3       0    113803  53.1000  C123        S  
6       0     17463  51.8625   E46        S  
10      1   PP 9549  16.7000    G6        S  
11      0    113783  26.5500  C103        S  
   index  PassengerId  Survived 

## Pivot tables
A pivot table groups information. The pivot_table method takes three parameters:
- index: the column to group by
- values: column(s) to get data from
- aggfunc: aggregate function to use

In [16]:
import numpy as np

# Average age of survivors
survived = data[data['Survived']==True]

# Remove null ages
survived = survived[survived['Age'].isnull() == False]

# Create pivot table to show average age for those who survived and check this against the survivors table
average_survived = survived['Age'].mean()

survived_pivot = data.pivot_table(index='Survived',values='Age',aggfunc=np.mean)
print("Average age of survivors: {}".format(average_survived))
print("Pivot table:")
print(survived_pivot.head())

Average age of survivors: 28.343689655172415
Pivot table:
Survived
0    30.626179
1    28.343690
Name: Age, dtype: float64


In [17]:
# You can also do this for multiple columns
multiple_pivot = data.pivot_table(index='Survived',values=['Age','Fare'],aggfunc=np.mean)
print(multiple_pivot)

                Age       Fare
Survived                      
0         30.626179  22.117887
1         28.343690  48.395408


In [18]:
# Remove all rows with missing values in 'Age' or 'Fare'
data_no_null_age_fare = data.dropna(axis=0,subset=['Age','Fare'])
print("Not null data: {}".format(data_no_null_age_fare.shape))
print("Original data: {}".format(data.shape))

Not null data: (714, 12)
Original data: (891, 12)


## Applying functions over a DataFrame
DataFrame.apply() will call a function for each column.
You can use 'axis=1' to send rows instead of columns.

In [27]:
def print_some_info(row):
    print("Info: Age:{},Fare:{},Cabin:{}".format(row['Age'],row['Fare'],row['Cabin']))
    
subset = data[:10]
subset.apply(print_some_info,axis=1)
print()

Info: Age:22.0,Fare:7.25,Cabin:nan
Info: Age:38.0,Fare:71.2833,Cabin:C85
Info: Age:26.0,Fare:7.925,Cabin:nan
Info: Age:35.0,Fare:53.1,Cabin:C123
Info: Age:35.0,Fare:8.05,Cabin:nan
Info: Age:nan,Fare:8.4583,Cabin:nan
Info: Age:54.0,Fare:51.8625,Cabin:E46
Info: Age:2.0,Fare:21.075,Cabin:nan
Info: Age:27.0,Fare:11.1333,Cabin:nan
Info: Age:14.0,Fare:30.0708,Cabin:nan



## More filtering

In [28]:
age_over_10 = data['Age'] > 10
age_under_30 = data['Age'] < 30
data_with_both = data[age_over_10 & age_under_30]
print("Original data: {}".format(data.shape))
print("Filtered data: {}".format(data_with_both.shape))

Original data: (891, 12)
Filtered data: (320, 12)


## Indexes
You can set a given column as index using: data.set_index('Column',drop=False)


In [33]:
# Display the numer of unique values in a column
print(data['Pclass'].value_counts().sort_index())

1    216
2    184
3    491
Name: Pclass, dtype: int64


In [39]:
# You can use apply to convert data
def string_to_age(age):
    if pd.isnull(age):
        return None
    return int(age)

int_ages = data['Age'].apply(string_to_age)
print("Original: {}".format(data['Age'].dtype))
print("Converted: {}".format(int_ages.dtype))

Original: float64
Converted: float64


In [48]:
import pandas as pd
titanic = pd.read_csv('titanic/train.csv')
# Group by and aggregate
titanic2 = titanic[['Age','Survived']].dropna() # Alternatively: fillna(value),f.ex df.mean()
titanic2_grouped = titanic2.groupby('Survived')
titanic2_grouped.agg(np.mean)
print(titanic2_grouped.head())

     Age  Survived
0   22.0         0
1   38.0         1
2   26.0         1
3   35.0         1
4   35.0         0
6   54.0         0
7    2.0         0
8   27.0         1
9   14.0         1
12  20.0         0


In [50]:
# To find correlations, we use corr()
titanic_correlations = titanic.corr()
np.fill_diagonal(titanic_correlations.values, -2) # Set self-correlations to lowest values so we can filter them out
print(titanic_correlations.head())

             PassengerId  Survived    Pclass       Age     SibSp     Parch  \
PassengerId    -2.000000 -0.005007 -0.035144  0.036847 -0.057527 -0.001652   
Survived       -0.005007 -2.000000 -0.338481 -0.077221 -0.035322  0.081629   
Pclass         -0.035144 -0.338481 -2.000000 -0.369226  0.083081  0.018443   
Age             0.036847 -0.077221 -0.369226 -2.000000 -0.308247 -0.189119   
SibSp          -0.057527 -0.035322  0.083081 -0.308247 -2.000000  0.414838   

                 Fare  
PassengerId  0.012658  
Survived     0.257307  
Pclass      -0.549500  
Age          0.096067  
SibSp        0.159651  


In [61]:
# Group by, simplest example
import pandas as pd
s1 = pd.Series([1,2,3,4,5])
s2 = pd.Series([10,20,30,40,50])
s3 = pd.Series(['a','b','c','d','e'])
s4 = pd.Series(['g1','g2','g1','g1','g2'])
df = pd.DataFrame({'s1':s1,'s2':s2,'s3':s3,'s4':s4})

group = df.groupby('s4')
agg = group.agg(np.mean) # Will only include columns that can be aggregated on mean (s1,s2) and drop the others(s3)
print(agg.head())

          s1         s2
s4                     
g1  2.666667  26.666667
g2  3.500000  35.000000
